In [1]:
import os
import random
import uuid
import numpy as np
from faker import Faker
from datetime import datetime
import time
import boto3
import json
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [4]:
# Initialize Faker with Spanish locale
fake = Faker('es_ES')

# Define geographical boundaries for Madrid
LAT_MIN, LAT_MAX = 40.312, 40.643
LON_MIN, LON_MAX = -3.888, -3.517

# Constants
max_weight_per_order = 50
max_vol_per_order = 100
status = "in_progress"
stream_name = 'OrderStreamForDispatching'  # Update with your Kinesis stream name

In [5]:
# Function to generate a random address in Madrid
def generate_random_address():
    street_name = fake.street_name()
    street_number = fake.building_number()
    lat = random.uniform(LAT_MIN, LAT_MAX)
    lon = random.uniform(LON_MIN, LON_MAX)
    address = f"{street_name}, {street_number}, Madrid, Spain"
    return {
        "address": address,
        "lat": lat,
        "lon": lon
    }

# Function to generate a random order
def generate_random_order(client_id):
    order_id = str(uuid.uuid4())
    volume = np.random.uniform(1, max_vol_per_order)
    weight = np.random.uniform(1, max_weight_per_order)
    n_objects = np.random.randint(1, 10)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Format the timestamp
    order = {
        "client_id": client_id,
        "location": generate_random_address(),
        "order_id": order_id,
        "order": {
            "n_objects": n_objects,
            "volume": volume,
            "weight": weight
        },
        "status": status,
        "timestamp": timestamp  # Use the formatted timestamp
    }
    print("Generated order:", order)  # Print the generated order
    return order

# Function to send the generated order to Kinesis
def send_order_to_kinesis(stream_name, order):
    order_data = json.dumps(order)
    
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=order_data.encode('utf-8'),
        PartitionKey=str(order['client_id'])  # Partition based on client_id
    )
    print(f"Order {order['order_id']} sent to Kinesis. Response: {response}")

# Function to generate orders at random intervals and send them to Kinesis
def order_stream_generator(stream_name):
    while True:
        client_id = random.randint(100000000000, 999999999999)
        order = generate_random_order(client_id)
        send_order_to_kinesis(stream_name, order)
        
        # Sleep for a random amount of time between 5 and 15 minutes
        sleep_time = random.randint(10, 50)  # 300 seconds (5 minutes) to 900 seconds (15 minutes)
        print(f"Sleeping for {sleep_time / 60} minutes")
        time.sleep(sleep_time)

In [6]:
# Start the order stream generator
order_stream_generator(stream_name)

Generated order: {'client_id': 902817992129, 'location': {'address': 'Vial de Carmelita Blanch, 300, Madrid, Spain', 'lat': 40.38193391535008, 'lon': -3.6687010949775436}, 'order_id': '6ef32c3b-abf0-430e-bbe8-2917605b02ed', 'order': {'n_objects': 5, 'volume': 39.42239747648925, 'weight': 26.430067873422175}, 'status': 'in_progress', 'timestamp': '2024-09-30 20:16:19'}
Order 6ef32c3b-abf0-430e-bbe8-2917605b02ed sent to Kinesis. Response: {'ShardId': 'shardId-000000000000', 'SequenceNumber': '49656295829544838090770373594158709623561425940723531778', 'ResponseMetadata': {'RequestId': 'e9fde06b-dff6-d393-b689-0d586792cf73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e9fde06b-dff6-d393-b689-0d586792cf73', 'x-amz-id-2': 'juA7hcGf2JhHnd5b7HSOMuLZ4yWQvEuYwfRWrCejGJ6zyFVB/pbG1eqgcZE129LwycohrAmU43/FRcirX2yuzSATZ2ZXrg0I', 'date': 'Mon, 30 Sep 2024 18:16:19 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 

KeyboardInterrupt: 